In [2]:
import pandas as pd

df = pd.read_csv('assets/data/datos.csv', delimiter=';')

#print(df.head())

print(df.columns)

Index(['Categoria', 'Enlace', 'Nombre', 'ImagenURL', 'Ingredientes',
       'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
       'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
       'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
       'Potasio(mg)'],
      dtype='object')


In [8]:
from langchain_openai import AzureOpenAIEmbeddings
import os
from dotenv import load_dotenv
import numpy as np
load_dotenv()

def similitud_cosenos(vect1:list, vect2:list)-> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    cosine_similarity = dot_product / (norm_A * norm_B)
    return cosine_similarity


embeddigs = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
    #api_version=os.getenv("api_version_openai_emb"),
    #deployment_name= os.getenv("deploy_name_emb")
)


In [10]:
import os
import pandas as pd
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
from sklearn.preprocessing import LabelEncoder

# Cargar variables de entorno
load_dotenv()

# Inicializar Azure OpenAI Embeddings
embeddings_model = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
    #api_version=os.getenv("api_version_openai_emb"),
    #deployment_name= os.getenv("deploy_name_emb")
)

# Cargar el dataset
df = pd.read_csv('assets/data/datos.csv', delimiter=';')  # Asegúrate de reemplazar con tu ruta

# Columnas de texto que queremos convertir
columnas_texto = ['Categoria', 'Nombre', 'Ingredientes']

# Aplicar embeddings a las columnas de texto y convertir a números
for col in columnas_texto:
    df[col + "_emb"] = df[col].astype(str).apply(lambda x: sum(embeddings_model.embed_query(x)))

# Codificar las categorías con números (Label Encoding)
label_encoder = LabelEncoder()
df["Categoria_Num"] = label_encoder.fit_transform(df["Categoria"].astype(str))

# Columnas numéricas originales + nuevas columnas transformadas
columnas_numericas = [
    'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
    'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
    'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
    'Potasio(mg)', 'Categoria_Num', 'Nombre_emb', 'Ingredientes_emb'
]

# Reemplazar NaN con 0
df[columnas_numericas] = df[columnas_numericas].fillna(0)

# Crear la nueva columna con la suma total
df["Suma_Total"] = df[columnas_numericas].sum(axis=1)

# Guardar el dataset con la nueva columna
df.to_csv("dataset_numerico.csv", index=False)

print("Dataset convertido y guardado en dataset_numerico.csv ✅")


Dataset convertido y guardado en dataset_numerico.csv ✅


In [14]:
dfN = pd.read_csv('assets/data/dataset_numerico.csv')

In [ ]:
dfN.columns

Index(['Categoria', 'Enlace', 'Nombre', 'ImagenURL', 'Ingredientes',
       'Calorias', 'Grasa Total(gr)', 'Colesterol(mg)', 'Sodio(mg)',
       'Carbohidrato Total(gr)', 'Proteina(gr)', 'Vitamina A(mcg)',
       'Vitamina C(mg)', 'Vitamina D(mcg)', 'Calcio(mg)', 'Hierro(mg)',
       'Potasio(mg)', 'Categoria_emb', 'Nombre_emb', 'Ingredientes_emb',
       'Categoria_Num', 'Suma_Total'],
      dtype='object')

In [20]:
import numpy as np

def similitud_cosenos(vect1:list, vect2:list)-> float:
    dot_product = np.dot(vect1, vect2)
    norm_A = np.linalg.norm(vect1)
    norm_B = np.linalg.norm(vect2)
    cosine_similarity = dot_product / (norm_A * norm_B)
    return cosine_similarity

In [53]:
import os
import pandas as pd
from langchain_openai import AzureOpenAIEmbeddings 
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity

# Cargar las variables de entorno desde un archivo .env
load_dotenv()

# Inicializar Azure OpenAI Embeddings
embeddings_model = AzureOpenAIEmbeddings(
    api_key=os.getenv("openai_embeddings_api_key"),
    base_url=os.getenv("openai_api_base_emb"),
    openai_api_type=os.getenv("openai_api_type"),
    # api_version=os.getenv("api_version_openai_emb"),  # Si tienes una versión específica
    # deployment_name=os.getenv("deploy_name_emb")  # Si tienes un deployment_name específico
)


# Función para generar embeddings utilizando el modelo text-embedding-3-large
def generate_embeddings(text, model="text-embedding-3-large"):
    response = embeddings_model.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

# Función para calcular la similitud de coseno entre dos embeddings
def similitud_cosenos(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Cargar el archivo CSV con las recetas o textos
df = pd.read_csv('assets/data/dataset_numerico.csv')  # Ajusta la ruta de tu archivo CSV


# Función para obtener la receta más similar al prompt
def obtener_receta_similar(prompt: str, df: pd.DataFrame) -> dict:
    # Generar embedding para el prompt
    prompt_embedding = generate_embeddings(prompt)

    # Calcular la similitud de coseno para cada receta en el DataFrame
    df['Similitud'] = df['Categoria_emb'].apply(lambda x: similitud_cosenos(prompt_embedding, x))

    # Buscar la receta más similar
    mejor_receta = df.loc[df['Similitud'].idxmax()]

    return mejor_receta

# Ejemplo de uso
prompt = "quiero una receta de pollo"
mejor_receta = obtener_receta_similar(prompt, df)

# Imprimir la receta más similar
print("La receta más similar es:", mejor_receta)



AttributeError: 'AzureOpenAIEmbeddings' object has no attribute 'embeddings'